<center><h1>Nelu-Kernelu</h1></center>

# The basics
The following NodeJS standard elements/classes are available to use within a cell:
* `console`
* `__dirname` and `__filename`
* `setImmediate`, `setInterval`, `setTimeout` with their `clearImmediate`, `clearInterval`, `clearInterval` (with a special _something_ discussed further down)
* `exports`, `module` and `require`
* `Buffer`
* `URL` and `URLSearchParams`
* `WebAssembly`
* `Promise` and `Error`

As you can see, and for security purposes, `process` and `global` are not part of this list.

Replacing `process` is a `kernel` object through which you can access the NodeJS's functionality and info.

### For instance ...

... you can print

In [1]:
kernel.print("I'm going to get printed underneath a code cell.")

I'm going to get printed underneath a code cell.

... and [format](https://nodejs.org/api/util.html#util_util_format_format_args) what you print

In [2]:
kernel.print("%d is a nice number while '%s' is a regular name.", 42, "John")

42 is a nice number while 'John' is a regular name.

> #### A note on `console.log`
> `console.log` does not print inside the notebook. Instead, it logs into the Notebook's process terminal like any normal console would expected to do.
> This means that this:
![console.log in](imgs/console_log_cell.png)
> will actually be visible here:
![console.log out](imgs/console_log_out.png)

... and find out who's currently running the kernel session, if you want.

In [3]:
kernel.userName

victor.adascalitei

### Feeling like reporting an error?

In [4]:
throw new Error('Here you go!');

Error: Here you go!

### What's a cell result? Simple ... whatever your last evaluation turns out to be

In [5]:
let a = 2
const b = 5

a*b

10

... except for _undefined_:

In [6]:
undefined

And no, if you were wondering, _null_ doesn't suffer from this:

In [7]:
null

null

### By the way ... a and b are not available anymore because they are scoped to the cell
_let_ and _const_ have this behaviour.

In [8]:
a

ReferenceError: a is not defined

In [9]:
b

ReferenceError: b is not defined

... but if you declare something like this (with _var_)

In [10]:
var c = 3;

... you can reference it later on

In [11]:
c

3

### Promises are resolved before their result is returned like so

In [12]:
Promise.resolve("I'm a result resolved by a Promise.")

I'm a result resolved by a Promise.

... this also works nested twice (_but don't do that_)

In [13]:
new Promise((accept, reject) => accept(new Promise((accept2, reject2) => accept2("2nd Promise resolved me"))))

2nd Promise resolved me

### How about versioning? How does that work?
We expose `kernel.version` to the user-cell which gives access to both a `name` and a `code`. 

The `name` is a dot separated, traditional, version name in the form of `x.y.z.w` where `x.y.z` is the actual supported [Jupyter Client protocol version](https://jupyter-client.readthedocs.io/en/stable/#) while `w` is the build number.
If you want a more numerical value (rather than a string), just go for the `code` which is computed as being `xyz000` + `w` where `x`, `y` and `z` are digits while `w` might be a number which, again is the source-build number.

In [14]:
kernel.print(kernel.version)

{ name: '5.3.0.1', code: 530001 }

### Want to render HTML? 
Return an object that has a _\_toHtml_ function which returns the HTML string you want rendered like so:

In [15]:
class RenderedAsHtml {
    _toHtml() { 
        return "<b>I'm a bolded text!</b>";
    }
}
new RenderedAsHtml()

I'm a bolded text!

... if _\_toHtml_ returns a Promise instead, we'll resolve it and render that

In [16]:
class RenderedAsPromisedHtml {
    _toHtml() { 
        return Promise.resolve("<b>I'm a <i>promised</i> bolded text!</b>");
    }
}
new RenderedAsPromisedHtml()

I'm a promised bolded text!

## Timers
They behave as you might expect with 2 caveats ([they're not covered by ECMAScript](https://stackoverflow.com/questions/8852198/settimeout-if-not-defined-in-ecmascript-spec-where-can-i-learn-how-it-works) so we allowed ourselves this insolence).
If a timer function is the last instruction the kernel sees in a cell, it blocks execution until it gets resolved meaning that:
* `setImmediate` - doesn't block execution (well, it does, but you never percieve it)
* `setTimeout` - blocks execution until the timer is triggered
* `setInterval` - blocks execution until `clearInterval` is triggered

In [17]:
setImmediate(() => kernel.print("Will print immediatelly without a noticeable delay."))

Will print immediatelly without a noticeable delay.

In [18]:
setTimeout(() => kernel.print("Will print after 2 seconds blocking execution in the meantime."), 2000)

Will print after 2 seconds blocking execution in the meantime.

In [19]:
let cnt = 0;
let timer;

kernel.print("Will print 3 ticks and then it will stop.");
timer = setInterval(() => {
    if (cnt == 3) clearInterval(timer);
    else {
        kernel.print(' Tick! ');
        cnt++;
    }
}, 1000)

// We need to return the timer so that it's the last thing evaluated in the cell
timer

Will print 3 ticks and then it will stop. Tick!  Tick!  Tick! 

Oh and whatever you return from an immediate/timeout timer callback will be the value that that cell resolves to:

In [20]:
setImmediate(() => {
    kernel.print("Printing from within the immediate callback");
    return "and returned this string."
})

Printing from within the immediate callback

and returned this string.

Of course, this is JS so we are still talking about an [event loop execution context](https://nodejs.org/en/docs/guides/event-loop-timers-and-nexttick/) which means that

In [21]:
setImmediate(() => kernel.print("This will print second ..."))
kernel.print("This will print first ...\n")

This will print first ...
This will print second ...

# The juicy stuff
## Comms
`comms` are a way to [relay data](https://jupyter-client.readthedocs.io/en/stable/messaging.html#custom-messages) between the kernel and Jupyter. The best example of their usage is the [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/latest/index.html) project. To make these mechanics possible, we export `kernel.commManager` which lets you work with them.

### Opening one
This is how you do in the fastest way possible:

In [22]:
let comm = kernel.commManager.newComm()

By default, they start-off with an empty data payload `{}` and it targets `jknb.comm` (the 'jknb' prefix was left for historical reasons). If you want to have a different target-name, a payload and/or a metadata even, you can use the more sophisticated way of opening a comm via the `kernel.commManager.newCommFor` method call like so: 

In [23]:
var myComm = kernel.commManager.newCommFor({
    targetName: 'my-target-name',
    initialData: {
        aVariable: 42
        // ... other props (if needed)
    },
    metaData: {
        version: 1
        // ... anything else?
    }
})

This has the consequence that a `comm_open` is immediatelly broadcast to `iopub` as dictated by the [Jupyter Client Specs](https://jupyter-client.readthedocs.io/en/stable/messaging.html#opening-a-comm). 
This is how it looks like if you want to feel brave:
![comm_open on wire](imgs/comm_open_on_wire.png)
### Sending data through a comm

In [24]:
myComm.send({
    anotherVariable: 24
    // add here whatever data needs to be sent
})

This ends up calling `comm_msg` and will look (on _the wire_) something like this:
![comm_msg on wire](imgs/comm_msg_on_wire.png)

### ... and receving some

In [25]:
myComm.on("message", ({ data }) => {
    // ... do whatever you want with 'data' inside the NodeJS kernel process
    kernel.print(JSON.stringify(data));
});

[object Object]

## Displays
Allow for [embedding more sofisticated results](https://jupyter-client.readthedocs.io/en/stable/messaging.html#display-data) into an output cell.

### Creating one
Is achieved by calling the `kernel.display` method passing in an instance of a class that extends `JupyterDisplayableMessage` and overrides the _\_toDisplay()_ method. 
\_toDisplay() needs to return a object that has the following structure:
```js
{
    "<mime_1>" : "value #1",
    "<mime_2>" : "value #2",
    // ...
    "<mime_n>" : "value #n"
}
```
For example, this will just display a `text/plain`:

In [26]:
class JustAText extends JupyterDisplayableMessage {
    _toDisplay() { 
        return { "text/plain": "I'm a plain text displayed value." };
    }
}
kernel.display(new JustAText());

I'm a plain text displayed value.

### _Bonus!_ Emulating a LabelView Jupyter Widget from NodeJs
> Note 1: You need to have `ipywidgets` installed to make use of them. [Here's](https://ipywidgets.readthedocs.io/en/latest/user_install.html) how you do it.  
> Note 2: The data payloads you see here on `comms` were reversed engineered from Jupyter Widgets

In [27]:
class LabelView extends JupyterDisplayableMessage {
    constructor(value) {
        super();
        this._value = value;
        this._wComm = kernel.commManager.newCommFor({
                targetName: 'jupyter.widget',
                initialData: {
                    state: {
                        _model_module: "@jupyter-widgets/controls",
                        _model_module_version: "1.4.0",
                        _model_name: "LabelModel",
                        _view_module: "@jupyter-widgets/controls",
                        _view_module_version: "1.4.0",
                        _view_name: "LabelView",
                        description: "?",
                        description_tooltip: "",
                        placeholder: "​",
                        value: this._value
                    },
                    buffer_paths: []
                },
                metaData: {
                    version: "2.0.0"
                }
            });
        
        this._wComm.on("message", ({ data }) => {
            if (data.method === 'request_state') {
                this._wComm.send({
                    method: "update",
                    state: {
                        _dom_classes: [],
                        _model_module: "@jupyter-widgets/controls",
                        _model_module_version: "1.4.0",
                        _model_name: "LabelModel",
                        _view_count: null,
                        _view_module: "@jupyter-widgets/controls",
                        _view_module_version: "1.4.0",
                        _view_name: "LabelView",
                        description: "",
                        description_tooltip: null,
                        placeholder: "​",
                        value: this._value
                    },
                    buffer_paths: []
                });
            }
        });
    }
    
    set value(newValue) {
        this._value = newValue;
        this._wComm.send({
            buffer_paths: [],
            method: "update",
            state: {
                value: this._value
            }
        })
    }
    
    _toDisplay() {
        return {
            "text/plain": `Label(value='${this._value}')`,
            "application/vnd.jupyter.widget-view+json": {
                version_major: 2,
                version_minor: 0,
                model_id: this._wComm.id
            }
        };
    }
}

// Instantiate it and display it
var label = new LabelView("This is how we start: $E=mc^2$");
kernel.display(label);

Label(value='This is how we start: $E=mc^2$')

#### and if you later go an change the label.value, guess what happens then?

In [28]:
label.value = `And this is how we end: ...`;

And this is how we end: ...

## Interrupting the kernel
I'm talking about clicking this:
![Kernel Interrupt Button](imgs/kernel_interrupt_button.png)
when you have this (don't ask why!):

In [29]:
while(1) {}

Error: Script execution was interrupted by `SIGINT`

#### or this (_never say never_)

In [30]:
setImmediate(() => { while(1); })

Error: Script execution was interrupted by `SIGINT`

Stops the execution without losing the context.
> Note! At the time of writing, there is still a bit of work to be done to fully support this, but we're getting there. 

This means that this:

In [31]:
c

3

will still be `3` provided that you executed the cell that set it
```js
var c = 3;
```

<center><h1>Happy JSing!</h1></center>